In [1]:
import pandas as pd

# === 1. Point to your files ===
file_fall   = '/Users/anjalikaarora/Desktop/Data Analysis Job/NCHA-III WEB FALL 2022 GEORGIA INSTITUTE OF TECHNOLOGY.xlsx'
file_spring = '/Users/anjalikaarora/Desktop/Data Analysis Job/NCHA-III WEB SPRING 2025 GEORGIA INSTITUTE OF TECHNOLOGY.xlsx'

# auto‑detect the single sheet in each
sheet_fall   = pd.ExcelFile(file_fall).sheet_names[0]
sheet_spring = pd.ExcelFile(file_spring).sheet_names[0]

# === 2. Load into DataFrames ===
df22 = pd.read_excel(file_fall,   sheet_name=sheet_fall)
df25 = pd.read_excel(file_spring, sheet_name=sheet_spring)

# === 3. Recode Gender Groups (same as Section E) ===
def recode_group(r):
    # N3Q67C: 1=woman, 2=man, others=GNC; N3Q67B: 1=no, 2=yes (trans flag)
    if r['N3Q67C']==2 and r['N3Q67B']==1:
        return 'Cis Men'
    if r['N3Q67C']==1 and r['N3Q67B']==1:
        return 'Cis Women'
    return 'Trans/GNC'

df22['Group'] = df22.apply(recode_group, axis=1)
df25['Group'] = df25.apply(recode_group, axis=1)

# === 4. Define Section F columns ===
lifetime_cols = [
    'N3Q22A1',  # Tobacco or nicotine delivery products
    'N3Q22A2',  # Alcoholic beverages
    'N3Q22A3',  # Cannabis (nonmedical)
    'N3Q22A4',  # Cocaine
    'N3Q22A5',  # Prescription stimulants (nonmedical)
    'N3Q22A6',  # Methamphetamine
    'N3Q22A7',  # Inhalants
    'N3Q22A8',  # Sedatives or sleeping pills (nonmedical)
    'N3Q22A9',  # Hallucinogens
    'N3Q22A10', # Heroin
    'N3Q22A11', # Prescription opioids (nonmedical)
]

recent_cols = [
    'N3Q22B1',  # past‑3‑mo tobacco
    'N3Q22B2',  # past‑3‑mo alcohol
    'N3Q22B3',  # past‑3‑mo cannabis
    'N3Q22B4',  # past‑3‑mo cocaine
    'N3Q22B5',  # past‑3‑mo stimulants
    'N3Q22B6',  # past‑3‑mo methamphetamine
    'N3Q22B7',  # past‑3‑mo inhalants
    'N3Q22B8',  # past‑3‑mo sedatives
    'N3Q22B9',  # past‑3‑mo hallucinogens
    'N3Q22B10', # past‑3‑mo heroin
    'N3Q22B11', # past‑3‑mo opioids
]

# === 5. Subset & tag waves ===
df_F22 = df22[lifetime_cols + recent_cols + ['Group']].copy()
df_F25 = df25[lifetime_cols + recent_cols + ['Group']].copy()

df_F22['Wave'] = 2022
df_F25['Wave'] = 2025

# === 6. (Optional) Combine both waves ===
df_F = pd.concat([df_F22, df_F25], ignore_index=True)

# === 7. Inspect shapes & head ===
print("Section F – Fall 2022:", df_F22.shape)
print(df_F22.head(), "\n")

print("Section F – Spring 2025:", df_F25.shape)
print(df_F25.head(), "\n")

# Now you’re ready to run your “ever used” vs. “used in last 3 months” analyses.


Section F – Fall 2022: (318, 24)
   N3Q22A1  N3Q22A2  N3Q22A3  N3Q22A4  N3Q22A5  N3Q22A6  N3Q22A7  N3Q22A8  \
0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
2      0.0      3.0      0.0      0.0      0.0      0.0      0.0      0.0   
3      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4      0.0      3.0      0.0      0.0      0.0      0.0      0.0      0.0   

   N3Q22A9  N3Q22A10  ...  N3Q22B4  N3Q22B5  N3Q22B6  N3Q22B7  N3Q22B8  \
0      0.0       0.0  ...      NaN      NaN      NaN      NaN      NaN   
1      0.0       0.0  ...      NaN      NaN      NaN      NaN      NaN   
2      0.0       0.0  ...      NaN      NaN      NaN      NaN      NaN   
3      0.0       0.0  ...      NaN      NaN      NaN      NaN      NaN   
4      0.0       0.0  ...      NaN      NaN      NaN      NaN      NaN   

   N3Q22B9  N3Q22B10  N3Q22B11      Group  Wave  
0      Na

In [19]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# === 1. File paths & sheet names ===
file_fall   = '/Users/anjalikaarora/Desktop/Data Analysis Job/NCHA-III WEB FALL 2022 GEORGIA INSTITUTE OF TECHNOLOGY.xlsx'
file_spring = '/Users/anjalikaarora/Desktop/Data Analysis Job/NCHA-III WEB SPRING 2025 GEORGIA INSTITUTE OF TECHNOLOGY.xlsx'
sheet_fall   = pd.ExcelFile(file_fall).sheet_names[0]
sheet_spring = pd.ExcelFile(file_spring).sheet_names[0]

# === 2. Load each wave ===
df22 = pd.read_excel(file_fall,   sheet_name=sheet_fall)
df25 = pd.read_excel(file_spring, sheet_name=sheet_spring)

# === 3. Recode gender into Overall/Cis Men/Cis Women/Trans GNC ===
# 1=M, 2=F, 3=Non‑binary/mismatch
df22['Group'] = df22['RSEX'].map({1:'Cis Women', 2:'Cis Men', 3:'Trans/GNC'})
df25['Group'] = df25['RSEX'].map({1:'Cis Women', 2:'Cis Men', 3:'Trans/GNC'})


# === 4. Section F variable lookup from codebook :contentReference[oaicite:0]{index=0}:contentReference[oaicite:1]{index=1} ===
ever_used = {
    'Tobacco or Nicotine':      'N3Q22A1',
    'Alcoholic Beverages':      'N3Q22A2',
    'Cannabis':                 'N3Q22A3',
    'Cocaine':                  'N3Q22A4',
    'Prescription Stimulants':  'N3Q22A5',
    'Methamphetamine':          'N3Q22A6',
    'Inhalants':                'N3Q22A7',
    'Sedatives/Sleeping Pills': 'N3Q22A8',
    'Hallucinogens':            'N3Q22A9',
    'Heroin':                   'N3Q22A10',
    'Prescription Opioids':     'N3Q22A11',
}

recent_used = {
    'Tobacco or Nicotine':      'N3Q22B1',
    'Alcoholic Beverages':      'N3Q22B2',
    'Cannabis':                 'N3Q22B3',
    'Cocaine':                  'N3Q22B4',
    'Prescription Stimulants':  'N3Q22B5',
    'Methamphetamine':          'N3Q22B6',
    'Inhalants':                'N3Q22B7',
    'Sedatives/Sleeping Pills': 'N3Q22B8',
    'Hallucinogens':            'N3Q22B9',
    'Heroin':                   'N3Q22B10',
    'Prescription Opioids':     'N3Q22B11',
}

groups = ['Overall','Cis Men','Cis Women','Trans/GNC']

def compute_table(labels, df22, df25, positive_codes):
    rows = []
    for label, col in labels.items():
        for grp in groups:
            d22 = df22 if grp=='Overall' else df22[df22['Group']==grp]
            d25 = df25 if grp=='Overall' else df25[df25['Group']==grp]
            n22, n25 = len(d22), len(d25)
            x22 = d22[col].isin(positive_codes).sum()
            x25 = d25[col].isin(positive_codes).sum()
            p22 = x22/n22*100 if n22 else np.nan
            p25 = x25/n25*100 if n25 else np.nan
            dppt = p25 - p22
            z, pval = proportions_ztest([x22, x25], [n22, n25])
            sig = 'Yes' if pval < 0.05 else 'No'
            rows.append({
                'Substance':    label,
                'Group':        grp,
                'Pct 2022 (%)': round(p22,1),
                'Pct 2025 (%)': round(p25,1),
                'Δ ppt':        round(dppt,1),
                'Significant?': sig
            })
    return pd.DataFrame(rows)

# === 5. Run analyses ===
#  3 = ever used (yes) :contentReference[oaicite:2]{index=2}:contentReference[oaicite:3]{index=3}
df_ever   = compute_table(ever_used,   df22, df25, positive_codes=[3])
#  2–5 = used in last 3 months :contentReference[oaicite:4]{index=4}:contentReference[oaicite:5]{index=5}
df_recent = compute_table(recent_used, df22, df25, positive_codes=[2,3,4,6])
print(df_recent.to_string(index=False))

# === 6. Print out ===
print("\n--- Ever Used (Lifetime) Changes ---")
print(df_ever.to_string(index=False))

print("\n--- Used in Last 3 Months Changes ---")
print(df_recent.to_string(index=False))


               Substance     Group  Pct 2022 (%)  Pct 2025 (%)  Δ ppt Significant?
     Tobacco or Nicotine   Overall          17.9          11.9   -6.0          Yes
     Tobacco or Nicotine   Cis Men          18.6          17.3   -1.3           No
     Tobacco or Nicotine Cis Women          16.7           7.3   -9.3          Yes
     Tobacco or Nicotine Trans/GNC          17.4           8.0   -9.4           No
     Alcoholic Beverages   Overall          70.1          61.8   -8.4          Yes
     Alcoholic Beverages   Cis Men          73.1          62.8  -10.3          Yes
     Alcoholic Beverages Cis Women          67.4          64.4   -3.0           No
     Alcoholic Beverages Trans/GNC          73.9          60.0  -13.9           No
                Cannabis   Overall          23.6          16.4   -7.2          Yes
                Cannabis   Cis Men          26.2          19.9   -6.3           No
                Cannabis Cis Women          18.8          14.1   -4.6           No
    

/opt/anaconda3/envs/practice_env/lib/python3.13/site-packages/statsmodels/stats/weightstats.py:792: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std


In [20]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# === 1. File paths & sheet names ===
file_fall   = '/Users/anjalikaarora/Desktop/Data Analysis Job/NCHA-III WEB FALL 2022 GEORGIA INSTITUTE OF TECHNOLOGY.xlsx'
file_spring = '/Users/anjalikaarora/Desktop/Data Analysis Job/NCHA-III WEB SPRING 2025 GEORGIA INSTITUTE OF TECHNOLOGY.xlsx'
sheet_fall   = pd.ExcelFile(file_fall).sheet_names[0]
sheet_spring = pd.ExcelFile(file_spring).sheet_names[0]

# === 2. Load each wave ===
df22 = pd.read_excel(file_fall,   sheet_name=sheet_fall)
df25 = pd.read_excel(file_spring, sheet_name=sheet_spring)

# === 3. Recode gender into Overall/Cis Men/Cis Women/Trans GNC ===
#    RSEX: 1=Female, 2=Male, 3=Non‑binary/mismatch
gender_map = {1: 'Cis Women', 2: 'Cis Men', 3: 'Trans/GNC'}
df22['Group'] = df22['RSEX'].map(gender_map)
df25['Group'] = df25['RSEX'].map(gender_map)

# === 4. Section F variable lookup from codebook ===
ever_used = {
    'Tobacco or Nicotine':      'N3Q22A1',
    'Alcoholic Beverages':      'N3Q22A2',
    'Cannabis':                 'N3Q22A3',
    'Cocaine':                  'N3Q22A4',
    'Prescription Stimulants':  'N3Q22A5',
    'Methamphetamine':          'N3Q22A6',
    'Inhalants':                'N3Q22A7',
    'Sedatives/Sleeping Pills': 'N3Q22A8',
    'Hallucinogens':            'N3Q22A9',
    'Heroin':                   'N3Q22A10',
    'Prescription Opioids':     'N3Q22A11',
}

recent_used = {
    'Tobacco or Nicotine':      'N3Q22B1',
    'Alcoholic Beverages':      'N3Q22B2',
    'Cannabis':                 'N3Q22B3',
    'Cocaine':                  'N3Q22B4',
    'Prescription Stimulants':  'N3Q22B5',
    'Methamphetamine':          'N3Q22B6',
    'Inhalants':                'N3Q22B7',
    'Sedatives/Sleeping Pills': 'N3Q22B8',
    'Hallucinogens':            'N3Q22B9',
    'Heroin':                   'N3Q22B10',
    'Prescription Opioids':     'N3Q22B11',
}

groups = ['Overall', 'Cis Men', 'Cis Women', 'Trans/GNC']

# === 5. Table‐building function (now including p‑value) ===
def compute_table(labels, df22, df25, positive_codes):
    rows = []
    for label, col in labels.items():
        for grp in groups:
            d22 = df22 if grp == 'Overall' else df22[df22['Group'] == grp]
            d25 = df25 if grp == 'Overall' else df25[df25['Group'] == grp]
            n22, n25 = len(d22), len(d25)
            x22 = d22[col].isin(positive_codes).sum()
            x25 = d25[col].isin(positive_codes).sum()
            p22 = x22 / n22 * 100 if n22 else np.nan
            p25 = x25 / n25 * 100 if n25 else np.nan
            dppt = p25 - p22
            z_stat, pval = proportions_ztest([x22, x25], [n22, n25])
            rows.append({
                'Substance':    label,
                'Group':        grp,
                'Pct 2022 (%)': round(p22, 1),
                'Pct 2025 (%)': round(p25, 1),
                'Δ ppt':        round(dppt, 1),
                'p‑value':      round(pval, 4),
                'Significant?': 'Yes' if pval < 0.05 else 'No'
            })
    return pd.DataFrame(rows)

# === 6. Run both analyses ===
#   Ever used = code “3”
df_ever   = compute_table(ever_used,   df22, df25, positive_codes=[3])
#   Used in last 3 months = codes 2,3,4,6
df_recent = compute_table(recent_used, df22, df25, positive_codes=[2,3,4,6])

# === 7. Print results ===
print("\n--- Ever Used (Lifetime) Changes ---")
print(df_ever.to_string(index=False))

print("\n--- Used in Last 3 Months Changes ---")
print(df_recent.to_string(index=False))



--- Ever Used (Lifetime) Changes ---
               Substance     Group  Pct 2022 (%)  Pct 2025 (%)  Δ ppt  p‑value Significant?
     Tobacco or Nicotine   Overall          29.9          21.9   -8.0   0.0130          Yes
     Tobacco or Nicotine   Cis Men          35.2          29.3   -5.9   0.2540           No
     Tobacco or Nicotine Cis Women          25.7          16.2   -9.5   0.0330          Yes
     Tobacco or Nicotine Trans/GNC          21.7          16.0   -5.7   0.6108           No
     Alcoholic Beverages   Overall          75.8          69.6   -6.2   0.0628           No
     Alcoholic Beverages   Cis Men          77.9          71.7   -6.2   0.1969           No
     Alcoholic Beverages Cis Women          74.3          69.6   -4.7   0.3476           No
     Alcoholic Beverages Trans/GNC          78.3          76.0   -2.3   0.8523           No
                Cannabis   Overall          39.0          32.8   -6.2   0.0804           No
                Cannabis   Cis Men        

/opt/anaconda3/envs/practice_env/lib/python3.13/site-packages/statsmodels/stats/weightstats.py:792: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std


# === A. Recovery Prevalence ===

In [41]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# === 1. Load data ===
file_fall   = '/Users/anjalikaarora/Desktop/Data Analysis Job/NCHA-III WEB FALL 2022 GEORGIA INSTITUTE OF TECHNOLOGY.xlsx'
file_spring = '/Users/anjalikaarora/Desktop/Data Analysis Job/NCHA-III WEB SPRING 2025 GEORGIA INSTITUTE OF TECHNOLOGY.xlsx'
sheet_fall   = pd.ExcelFile(file_fall).sheet_names[0]
sheet_spring = pd.ExcelFile(file_spring).sheet_names[0]

df22 = pd.read_excel(file_fall, sheet_name=sheet_fall)
df25 = pd.read_excel(file_spring, sheet_name=sheet_spring)

# === 2. Recode gender groups ===
gender_map = {1: 'Cis Women', 2: 'Cis Men', 3: 'Trans/GNC'}
df22['Group'] = df22['RSEX'].map(gender_map)
df25['Group'] = df25['RSEX'].map(gender_map)

# === 3. Recency code mapping ===
recency_codes = {
    1: 'Never',
    2: 'Within last 2 weeks',
    3: 'More than 2 weeks ago but ≤30 days',
    4: 'More than 30 days ago but ≤3 months',
    5: 'More than 3 months ago but ≤12 months',
    6: 'More than 12 months ago'
}

# === 4. Compute recency differences for alcohol ===
rows = []
for code, label in recency_codes.items():
    for grp in ['Overall', 'Cis Men', 'Cis Women', 'Trans/GNC']:
        d22 = df22 if grp == 'Overall' else df22[df22['Group'] == grp]
        d25 = df25 if grp == 'Overall' else df25[df25['Group'] == grp]
        valid22 = d22['N3Q25A'].isin(recency_codes)
        valid25 = d25['N3Q25A'].isin(recency_codes)
        n22, n25 = valid22.sum(), valid25.sum()
        x22 = (d22['N3Q25A'] == code).sum()
        x25 = (d25['N3Q25A'] == code).sum()
        p22 = x22 / n22 * 100 if n22 else np.nan
        p25 = x25 / n25 * 100 if n25 else np.nan
        dp  = p25 - p22
        _, pval = proportions_ztest([x22, x25], [n22, n25])
        rows.append({
            'Group':          grp,
            'Category':       label,
            'Pct 2022 (%)':   round(p22, 1),
            'Pct 2025 (%)':   round(p25, 1),
            'Δ ppt':          round(dp, 1),
            'p‑value':        round(pval, 4),
            'Significant?':   'Yes' if pval < 0.05 else 'No'
        })

alc_diff = pd.DataFrame(rows)

# === 5. Pivot wide ===
wide_alc = alc_diff.pivot(
    index='Group',
    columns='Category',
    values=['Pct 2022 (%)','Pct 2025 (%)','Δ ppt','p‑value','Significant?']
)

# Display the wide-format table
wide_alc.shape
wide_alc.to_csv('wide_alc.csv', index=True)


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# 1. File paths & load
file_fall   = 'NCHA-III WEB FALL 2022 GEORGIA INSTITUTE OF TECHNOLOGY.xlsx'
file_spring = 'NCHA-III WEB SPRING 2025 GEORGIA INSTITUTE OF TECHNOLOGY.xlsx'
sheet_fall   = pd.ExcelFile(file_fall).sheet_names[0]
sheet_spring = pd.ExcelFile(file_spring).sheet_names[0]
df22 = pd.read_excel(file_fall,   sheet_name=sheet_fall)
df25 = pd.read_excel(file_spring, sheet_name=sheet_spring)

# 2. Recode gender
gender_map = {1:'Cis Women', 2:'Cis Men', 3:'Trans/GNC'}
df22['Group'] = df22['RSEX'].map(gender_map)
df25['Group'] = df25['RSEX'].map(gender_map)

# 3. Filter to those who drank in last 12 months (codes 2–5)
recent_alc_codes = [2,3,4,5]
df22_alc = df22[df22['N3Q25A'].isin(recent_alc_codes)]
df25_alc = df25[df25['N3Q25A'].isin(recent_alc_codes)]

# 4. Define the 12 consequences + a “two or more” summary
consequences = {
    'Regretted something':              'N3Q29A',
    'Blackout (large gap)':             'N3Q29B',
    'Brownout (short gap)':             'N3Q29C',
    'Trouble with police':              'N3Q29D',
    'Trouble with authorities':         'N3Q29E',
    'Someone had sex without my consent':'N3Q29F',
    'Had sex without their consent':    'N3Q29G',
    'Unprotected sex':                  'N3Q29H',
    'Physically injured myself':        'N3Q29I',
    'Physically injured another':       'N3Q29J',
    'Seriously considered suicide':     'N3Q29K',
    'Needed medical help':              'N3Q29L',
}

rows = []
groups = ['Overall','Cis Men','Cis Women','Trans/GNC']

for label, var in consequences.items():
    for grp in groups:
        d22 = df22_alc if grp=='Overall' else df22_alc[df22_alc['Group']==grp]
        d25 = df25_alc if grp=='Overall' else df25_alc[df25_alc['Group']==grp]
        n22, n25 = len(d22), len(d25)
        x22 = (d22[var]==2).sum()   # code 2 = “Yes”
        x25 = (d25[var]==2).sum()
        p22 = x22/n22*100 if n22 else np.nan
        p25 = x25/n25*100 if n25 else np.nan
        dp  = p25 - p22
        _, pval = proportions_ztest([x22, x25], [n22, n25])
        rows.append({
            'Category':     label,
            'Group':        grp,
            'Pct 2022 (%)': round(p22,1),
            'Pct 2025 (%)': round(p25,1),
            'Δ ppt':        round(dp,1),
            'p‑value':      round(pval,4),
            'Significant?': 'Yes' if pval<0.05 else 'No'
        })

# 4b. “Two or more” of the above
for grp in groups:
    d22 = df22_alc if grp=='Overall' else df22_alc[df22_alc['Group']==grp]
    d25 = df25_alc if grp=='Overall' else df25_alc[df25_alc['Group']==grp]
    n22, n25 = len(d22), len(d25)
    count22 = (d22[list(consequences.values())]==2).sum(axis=1).ge(2).sum()
    count25 = (d25[list(consequences.values())]==2).sum(axis=1).ge(2).sum()
    p22 = count22/n22*100
    p25 = count25/n25*100
    dp  = p25 - p22
    _, pval = proportions_ztest([count22, count25], [n22, n25])
    rows.append({
        'Category':     'Two or more',
        'Group':        grp,
        'Pct 2022 (%)': round(p22,1),
        'Pct 2025 (%)': round(p25,1),
        'Δ ppt':        round(dp,1),
        'p‑value':      round(pval,4),
        'Significant?': 'Yes' if pval<0.05 else 'No'
    })

# 5. Build DataFrame & pivot wide
df_cons = pd.DataFrame(rows)
wide_cons = df_cons.pivot(
    index='Category',
    columns='Group',
    values=['Pct 2022 (%)','Pct 2025 (%)','Δ ppt','p‑value','Significant?']
)
# flatten column index
wide_cons.columns = [f"{grp} {metric}" for metric,grp in wide_cons.columns]

# 6. Save to CSV
wide_cons.to_csv('alcohol_consequences_diff.csv', index=True)

print("Saved table to alcohol_consequences_diff.csv")
